## Ingest Vectors to Index

#### Set environment variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

ai_search_endpoint  = os.getenv("AI_SEARCH_ENDPOINT")
if ai_search_endpoint is None or ai_search_endpoint == "":
    print("AI_SEARCH_ENDPOINT environment variable not set.")
    exit(1)

ai_search_api_version  = os.getenv("AI_SEARCH_API_VERSION")
if ai_search_api_version is None or ai_search_api_version == "":
    print("AI_SEARCH_API_VERSION environment variable not set.")
    exit(1)

ai_search_key  = os.getenv("AI_SEARCH_KEY")
if ai_search_key is None or ai_search_key == "":
    print("AI_SEARCH_KEY environment variable not set.")
    exit(1)

index_prefix = os.getenv("INDEX_PREFIX")
if index_prefix is None or index_prefix == "":
    print("INDEX_PREFIX environment variable not set.")
    exit(1)

grocery_index = f"{index_prefix}_grocery"
grocery_image_index = f"{index_prefix}_grocery_image"

#### Ingest grocery vector

In [2]:
import requests
import json
import pandas as pd
from azure.search.documents import SearchClient  
from azure.core.credentials import AzureKeyCredential

text_df = pd.read_json('data/grocery_embeddings.json') 
search_client = SearchClient(ai_search_endpoint, grocery_index, AzureKeyCredential(ai_search_key))

batch_size = 10
total_records = text_df.shape[0]
fields = text_df.columns.to_numpy()
text_df['id'] = text_df['id'].astype(str)

records = []

for index, row in text_df.iterrows():
    record = {}
    for field in fields:
                record[field] = row[field]

    records.append(
        record
    )

    if index % batch_size == 0 or (index+1 == total_records):
        try:
            result = search_client.upload_documents(documents=records)
            records = []
        except:
              print(records)

#### Ingest grocery image vector

In [3]:
import requests
import json
import pandas as pd
from azure.search.documents import SearchClient  
from azure.core.credentials import AzureKeyCredential

text_df = pd.read_json('data/images_embeddings.json') 
search_client = SearchClient(ai_search_endpoint, grocery_image_index, AzureKeyCredential(ai_search_key))

batch_size = 10
total_records = text_df.shape[0]
fields = text_df.columns.to_numpy()
text_df['id'] = text_df['id'].astype(str)

records = []

for index, row in text_df.iterrows():
    record = {}
    for field in fields:
                record[field] = row[field]

    records.append(
        record
    )

    if index % batch_size == 0 or (index+1 == total_records):
        try:
            result = search_client.upload_documents(documents=records)
            records = []
        except:
              print(records)